In [1]:
import sys
import json
import os
import requests
import random
from datetime import datetime
import time

import pandas as pd
import numpy as np
import sklearn.linear_model

# host = ""

def load_digits(digits_location, digits_filename = "train.data", norm=True):
    digits_path = digits_location + "/" + digits_filename
    print("Source file: %s" % digits_path)
    df = pd.read_csv(digits_path, sep=",", header=None)
    data = df.values
    print("Number of image files: %d" % len(data))
    y = data[:,0]
    X = data[:,1:]
    Z = X
    if norm:
        mu = np.mean(X,0)
        sigma = np.var(X,0)
        Z = (X - mu) / np.array([np.sqrt(z) if z > 0 else 1. for z in sigma])
    return Z, y

def filter_data(data):
    cx, cy = data
    binary_x = []
    binary_y = []
    for i in range(len(cy)):
        if cy[i] == 3:
            binary_x.append(cx[i,:])
            binary_y.append(1.0)
        elif cy[i] == 6:
            binary_x.append(cx[i,:])
            binary_y.append(0.0)
    return np.array(binary_x), np.array(binary_y)
            

def mnist_update(host, uid, x, y):
    url = "http://%s:1337/update" % host
    req_json = json.dumps({'uid': uid, 'input': list(x), 'label': y})
    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

def mnist_prediction(host, uid, x):
    url = "http://%s:1337/predict" % host
    req_json = json.dumps({'uid': uid, 'input': list(x)})
    headers = {'Content-type': 'application/json'}
    # x_str = ", ".join(["%d" % a for a in x])
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    pred = float(r.text.split(":")[1].strip())
    return (pred, latency)



def send_updates(num_updates):
    uid = 0
    for i in range(num_updates):
        example_num = np.random.randint(0, len(test_y))
        mnist_update(host, uid, test_x[example_num], float(test_y[example_num]))
    

def run_serving_workload():
    uid = 0
    while True:
        correct = 0
        false_pos = 0
        false_neg = 0
        latencies = []
        true_pos = 0
        true_neg = 0
        for i in range(100):
            example_num = np.random.randint(0, len(test_y))
            correct_y = float(test_y[example_num])
            pred_y, latency = mnist_prediction(host, uid, test_x[example_num])
            if correct_y == pred_y:
                if correct_y == 0:
                    true_neg += 1
                elif correct_y == 1:
                    true_pos += 1
                correct += 1
            elif correct_y == 0 and pred_y == 1:
                false_pos += 1
            elif correct_y == 1 and pred_y == 0:
                false_neg += 1
            else:
                print "predicted: {p}, correct: {c}".format(p=pred_y, c=correct_y)
            latencies.append(latency)
        print "Accuracy: {cor}, false positives: {fp}, false negatives: {fn}, true positives: {tp}, true negatives: {tn}".format(
            cor = float(correct)/100.0,
            fp=float(false_pos)/100.0,
            fn=float(false_neg)/100.0,
            tp=float(true_pos)/100.0,
            tn=float(true_neg)/100.0)
        print "Mean latency: {lat} ms".format(lat = np.mean(latencies))
            
    


test_x, test_y = filter_data(load_digits(os.path.expanduser("~/model-serving/data/mnist_data"), digits_filename="test.data"))


Source file: /Users/crankshaw/model-serving/data/mnist_data/test.data
Number of image files: 10000


In [2]:
host = "ec2-52-53-148-130.us-west-1.compute.amazonaws.com"
send_updates(50)
#run_serving_workload()

'Update scheduled', 122.857000 ms
'Update scheduled', 107.626000 ms
'Update scheduled', 99.800000 ms
'Update scheduled', 550.399000 ms
'Update scheduled', 145.544000 ms
'Update scheduled', 107.906000 ms
'Update scheduled', 115.918000 ms
'Update scheduled', 103.876000 ms
'Update scheduled', 95.218000 ms
'Update scheduled', 102.117000 ms
'Update scheduled', 191.722000 ms
'Update scheduled', 216.782000 ms
'Update scheduled', 146.168000 ms
'Update scheduled', 114.821000 ms
'Update scheduled', 157.077000 ms
'Update scheduled', 177.265000 ms
'Update scheduled', 106.434000 ms
'Update scheduled', 170.149000 ms
'Update scheduled', 129.698000 ms
'Update scheduled', 104.087000 ms
'Update scheduled', 96.141000 ms
'Update scheduled', 101.230000 ms
'Update scheduled', 169.341000 ms
'Update scheduled', 189.617000 ms
'Update scheduled', 167.755000 ms
'Update scheduled', 147.963000 ms
'Update scheduled', 141.070000 ms
'Update scheduled', 117.512000 ms
'Update scheduled', 100.441000 ms
'Update scheduled